In [ ]:
    @staticmethod
    def process_tools(tools):
        """
        Process a list of tools into Gemini API compatible schemas.
        
        Args:
            tools: List of tools that can be:
                - Functions (schema extracted automatically)
                - Tuples of (function, params_model, [response_model])
                
        Returns:
            Tuple of (processed_tools, tool_callables)
            - processed_tools: List of tool schemas
            - tool_callables: Dict mapping tool names to callable functions
        """
        import inspect
        
        processed_tools = []
        tool_callables = {}
        
        if not tools:
            return processed_tools, tool_callables
        
        for tool in tools:
            if isinstance(tool, tuple) and len(tool) >= 2 and callable(tool[0]):
                # Case: (function, params_model, [response_model]) tuple
                function = tool[0]
                params_model = tool[1]
                response_model = tool[2] if len(tool) > 2 else None
                
                # Convert Pydantic models to schema
                func_name = function.__name__
                func_description = inspect.getdoc(function) or ""
                
                params_schema = GeminiUtils.pydantic_to_schema(params_model)
                
                # Create schema
                schema = {
                    "name": func_name,
                    "description": func_description,
                    "parameters": params_schema
                }
                
                # Add response schema if provided
                if response_model:
                    response_schema = GeminiUtils.pydantic_to_schema(response_model)
                    schema["response"] = response_schema
                
                processed_tools.append(schema)
                tool_callables[func_name] = function
                
            elif callable(tool):
                # Case: Plain function
                schema = GeminiUtils.python_to_function(tool)
                processed_tools.append(schema)
                tool_callables[schema["name"]] = tool
                
            else:
                logger.warning(f"Ignoring unsupported tool type: {type(tool)}")
        
        return processed_tools, tool_callables

In [ ]:
    @classmethod
    def init(
        cls,
        app_id: str,
        env: Optional[str] = None,
        model_name: Optional[str] = None,
        temperature: Optional[float] = None,
        reasoning_effort: Optional[Literal["low", "medium", "high"]] = None,
        tools: Optional[List[Union[Callable, Tuple[Callable, Type[BaseModel], Optional[Type[BaseModel]]]]]] = None,
        provider_type: Literal["cms", "dsml"] = "cms",
        log_level: Optional[Literal["INFO", "DEBUG"]] = "INFO",
        **kwargs,
    ) -> "LLM":
        """
        Create and initialize an LLM adapter with default services.
        
        Args:
            app_id: Application identifier
            env: Environment name
            model_name: Name of the language model to use
            temperature: Temperature parameter for generation
            reasoning_effort: Reasoning effort level
            tools: Tools that can be used by the model. Can be:
                - Functions (schema extracted automatically)
                - Tuples of (function, params_model, response_model)
            provider_type: Default provider type to use
            log_level: Logging level
            **kwargs: Additional configuration options
            
        Returns:
            Configured LLM adapter
        """
        # Get preferences
        preferences = get_preferences(
            inference_model_name=model_name,
            temperature=temperature,
            reasoning_effort=reasoning_effort,
            **kwargs,
        )
        
        # Process tools to standardized format using GeminiUtils
        processed_tools, tool_callables = GeminiUtils.process_tools(tools)
        
        if provider_type == "cms":
            # Create CMS service with processed tools
            cms = ConversationManagementService(
                app_id=app_id,
                env=env,
                preferences=preferences,
                tools=processed_tools,
                log_level=log_level,
                **kwargs,
            )
            
            # Register tool implementations
            for name, func in tool_callables.items():
                cms.tool_callables[name] = func
            
            # Create LLM instance
            return cls(
                app_id=app_id,
                env=env,
                cms=cms,
                provider_type=provider_type,
            )
        elif provider_type == "dsml":
            # DSML support would be implemented here
            raise NotImplementedError("DSML provider not yet implemented")
        else:
            raise ValueError(f"Unknown provider type: {provider_type}")